<a href="https://colab.research.google.com/github/wdconinc/practical-computing-for-scientists/blob/master/Lectures/lecture21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture #21

In [0]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import numpy.matlib as ml

##In our last episode

* 1st order ODEs
  * the Runge-Kutta 4th order method coded

## Solving 1st order Ordinary Differential Equations (ODEs)

### 4th order Runge-Kutta

Now that we know how to derive the general Runge-Kutta methods, we could (given enough time) derive the most commonly used Runge-Kutta method of 4th order. This method has a truncation error of $\mathcal{O}(h^5)$ per step.
￼
For the 4th order Runge-Kutta method we use:
$$ y_{n+1} = y_{n} + h\sum_{i=1}^4 b_i k_i $$
with the $k_i$ evaluated at
$$ k_1 = f\big(t_n + c_1 h, y_n\big) $$
$$ k_2 = f\big(t_n + c_2 h, y_n + h (a_{21} k_1)\big) $$
$$ k_3 = f\big(t_n + c_3 h, y_n + h (a_{31} k_1 + a_{32} k_2)\big) $$
$$ k_4 = f\big(t_n + c_4 h, y_n + h (a_{41} k_1 + a_{42} k_2 + a_{43} k_3)\big) $$
and with the coefficients
$$ b_1 = \frac{1}{6} \quad b_2 = \frac{1}{3} \quad b_3 = \frac{1}{3} \quad b_4 = \frac{1}{6} $$
$$ c_1 = 0 \quad c_2 = \frac{1}{2} \quad c_3 = \frac{1}{2} \quad c_4 = 1 $$
$$ a_{21} = \frac{1}{2} \quad a_{32} = \frac{1}{2} \quad a_{43} = 1 $$


In [0]:
def solve_rk4(f, t, y0):
    y = np.zeros_like(t)
    y[0] = y0
    for i in range(0, len(t) - 1):
        h = t[i+1] - t[i]
        k1 = h*f(y[i], t[i])
        k2 = h*f(y[i] + k1/2, t[i] + h/2)
        k3 = h*f(y[i] + k2/2, t[i] + h/2)
        k4 = h*f(y[i] + k3, t[i] + h)
        y[i+1] = y[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4)
    return y

In [0]:
f1 = lambda y, t: -y*t
f1_exsol = lambda t: np.exp(-0.5*t**2)
t = np.linspace(0, 2, 25)
plt.plot(t,f1_exsol(t))
y0 = 1
y1_rk4 = solve_rk4(f1, t, y0)
plt.plot(t, y1_rk4, "--og", ms = 4)

In [0]:
plt.plot(t, f1_exsol(t) - y1_rk4)

In [0]:
f2 = lambda y,t: -np.sin(t)
f2_exact = lambda t: np.cos(t)
y0 = 1
t = np.linspace(0,60,120)
plt.plot(t, f2_exact(t))
Y2solution = solve_rk4(f2, t, y0)
plt.plot(t, Y2solution, '--og', ms = 3)

In [0]:
plt.plot(t, f2_exact(t) - Y2solution)

### 4th-5th order Runge-Kutta: adaptive steps

There is one more improvement we can make to our Runge-Kutta method: *adaptive step sizes*. Instead of specifying the set of points ourselves in advance, let's let the algorithm figure it out!

We could use an approach similar to the bisection algorithm or the trapezoid rule: we start with a step size $h$ and we keep reducing the step size $h$ to $h/2$ until the difference becomes smaller than a preset tolerance. However, this is inefficient and requires that we recalculate the entire problem multiple times.

Instead we choose to use a specific set of 4th and 5th order Runge-Kutta coefficients which have a lot of overlap. For each $y_{n+1}$ which we calculate with the 4th order Runge-Kutta coefficients, we *also* calculate $z_{n+1}$ with the 5th order Runge-Kutta coefficients.

Since the two orders have a truncation error that differs by 1 order of $h$, we can use the difference to determine a better value for $h$ given some tolerance.

#### 4th-5th order Runge-Kutta-Fehlberg

For the 4th-5th order Runge-Kutta-Fehlberg method we evaluate the $k_i$ evaluated at
$$ k_1 = f\big(t_n, y_n\big) $$

$$ k_2 = f\Big(t_n + \frac{1}{4} h_n, y_n + \frac{1}{4} h_n k_1\Big) $$

$$ k_3 = f\Big(t_n + \frac{3}{8} h_n, y_n + h_n\big(\frac{3}{32} k_1 + \frac{9}{32} k_2\big)\Big) $$

$$ k_4 = f\Big(t_n + \frac{12}{13} h_n, y_n + h_n\big(\frac{1932}{2197} k_1 - \frac{7200}{2197} k_2 + \frac{7296}{2197} k_3\big)\Big) $$

$$ k_5 = f\Big(t_n + h_n, y_n + h_n\big(\frac{439}{216} k_1 - 8 k_2 + \frac{3680}{513} k_3 - \frac{845}{4104} k_4\big)\Big) $$

$$ k_6 = f\Big(t_n + \frac{1}{2} h_n, y_n + h_n\big(-\frac{8}{27} k_1 +2 k_2 - \frac{3544}{2565} k_3 + \frac{1859}{4104} k_4 - \frac{11}{40} k_5\big)\Big) $$

We can then determine the 4th order point
$$ y_{n+1} = y_{n} + h_n\Big( \frac{25}{216} k_1 + \frac{1408}{2565} k_3 + \frac{2197}{4104} k_4 - \frac{1}{5} k_5 \Big) $$
and the 5th order point
$$ z_{n+1} = y_{n} + h_n\Big( \frac{16}{135} k_1 + \frac{6656}{12825} k_3 + \frac{28561}{56430} k_4 - \frac{9}{50} k_5 - \frac{2}{55} k_6\Big) $$

Now we can compare the determinations for $y_{n+1}$ and for $z_{n+1}$, and detemine the optimal step size $h_{n+1}$ for a precision tolerance $tol$:
$$ h_{n+1} = h_n \sqrt[4]{\frac{h_n \, tol}{2 |z_{n+1} - y_{n+1}|}} $$

In [0]:
def solve_rk45(f, t0, t1, y0, tol = 1e-6, maxn = 200):
    t = np.array([t0], dtype=np.float64)
    y = np.array([y0], dtype=np.float64)
    z = np.array([y0], dtype=np.float64)
    h = (t1 - t0) / maxn # initial step
    i = 0
    while t[i] < t1 and i < maxn:
        t = np.append(t, 0)
        y = np.append(y, 0)
        z = np.append(z, 0)
        t[i+1] = t[i] + h

        k1 = h*f(y[i], t[i])
        k2 = h*f(y[i] + k1/4, t[i] + h/4)
        k3 = h*f(y[i] + 3*k1/32 + 9*k2/32, t[i] + 3*h/8)
        k4 = h*f(y[i] + 1932*k1/2197 - 7200*k2/2197 + 7296*k3/2197, t[i] + 12*h/13)
        k5 = h*f(y[i] + 439*k1/216 - 8*k2 + 3680*k3/513 - 845*k4/4104, t[i] + h)
        k6 = h*f(y[i] - 8*k1/27 + 2*k2 - 3544*k3/2565 + 1859*k4/4104 - 11*k5/40, t[i] + h/2)
        y[i+1] = y[i] + 25*k1/216 + 1408*k3/2565 + 2197*k4/4104 - k5/5
        z[i+1] = y[i] + 16*k1/135 + 6656*k3/12825 + 28561*k4/56430 - 9*k5/50 + 2*k6/55

        if math.fabs(z[i+1] - y[i+1]) > 0:
            h = h * (h * tol / 2 / math.fabs(z[i+1] - y[i+1]))**0.25
            
        i = i+1
            
    if i == maxn:
        return ArithmeticError("Failed to converge")

    return t, y

In [0]:
f1 = lambda y, t: -y*t
f1_exsol = lambda t: np.exp(-0.5*t**2)
t0 = 0
t1 = 2
y0 = 1
t = np.linspace(t0, t1, 25)
plt.plot(t,f1_exsol(t))
t1solution, y1_rk45 = solve_rk45(f1, t0, t1, y0)
plt.plot(t1solution, y1_rk45, "--og", ms = 4)

In [0]:
print("Average step size %f in %d steps" % ((t1-t0)/len(t1solution), len(t1solution)))
plt.plot(t1solution, f1_exsol(t1solution) - y1_rk45)

In [0]:
f2 = lambda y,t: -np.sin(t)
f2_exact = lambda t: np.cos(t)
t0 = 0
t1 = 60
y0 = 1
t = np.linspace(t0, t1, 120)
plt.plot(t,f2_exact(t))
t2solution, y2solution = solve_rk45(f2, t0, t1, y0)
plt.plot(t2solution, y2solution, '--og', ms = 3)

In [0]:
print("Average step size %f in %d steps" % ((t1-t0)/len(t2solution), len(t2solution)))
plt.plot(t2solution, f2_exact(t2solution) - y2solution)

#### Variations in step size based on the behavior of the function

In [0]:
f3 = lambda y,t: -np.sin(t**2)
t0 = 0
t1 = 10
y0 = 1
t3solution, y3solution = solve_rk45(f3, t0, t1, y0)
plt.plot(t3solution, y3solution, '-og', ms = 4)

## Beyond 1st order ODE: Solving Hermite's Equation

Hermite's differential equation is:

$$ y'' - 2ty' + 2ny = 0 $$

This is 2nd order ODE. To solve it numerically we introduce a new variable, the velocity - $v$. Then the 2nd order equation can be written as 2 first order equations:

$ y' = v $

$ v' = 2tv-2ny$

We'll now treat $y,v$ as 0th and 1st elements of an array. We'll adopt a general notation $y_{ij}$ where $j$ labels the variable and $i$ the timestep. So, for a timestep $i$ the above two equations would be

$ y_{i0}' = y_{i1} $

$ y_{i1}' = 2t_{i} y_{i1} -2ny_{i0}$  

In the vectorized RK4 solver below $y_{ij}$ is the element `y[i,j]` of the 2D array.


In [0]:
#### hermite function with n, default order 2
def fhn(y, t, n = 2):
    yprime = np.zeros_like(y)
    yprime[0] = y[1]
    yprime[1] = 2*t*y[1] - 2*n*y[0]
    return yprime

As a given, we know the initial conditions at $x = 0$: $y(0) = -2$ and $y'(0) = 0$.

In [0]:
y0 = [-2,0]

###Playing around with arrays

This is the sort of testing I had to do in order to vectorize `solve_rk4`

In [0]:
v = np.zeros((4,2))
print(v)
w = np.asarray([1,2])
print(w)
v[1] = w
print(v)

###Vectorizing `solve_rk4`

In [0]:
def vsolve_rk4(f,t,y0):
    y0 = np.asarray(y0)
    t = np.asarray(t)
    y = np.zeros((len(t),len(y0)))
    y[0] = y0
    for i in range(0, len(t) - 1):
        h = t[i+1] - t[i]
        k1 = h*f(y[i], t[i])
        k2 = h*f(y[i] + k1/2, t[i] + h/2)
        k3 = h*f(y[i] + k2/2, t[i] + h/2)
        k4 = h*f(y[i] + k3, t[i] + h)
        y[i+1] = y[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4)
    return y

In [0]:
n = 2
t = np.linspace(0, 2, 50)
fh = lambda y, t: fhn(y, t, n)
H = sp.special.hermite(n)
plt.plot(t, H(t))
y = vsolve_rk4(fh, t, y0)
plt.plot(t, y[:,0], "or", ms = 3)

In [0]:
plt.plot(H(t) - y[:,0])

###Physical pendulum

The ODE for a pendulum is:

$$\theta'' + \frac{mgh}{I}\sin\theta = 0$$

This is the _physical_ pendulum, in contrast to the case $\sin\theta\approx\theta$ which is the _idealized_ or small angle pendulum.  I solve it below for a meterstick rotated around its end.  

Idealized pendula undergo simple harmonic motion with $T_{ideal}=2\pi/\sqrt{mgh/I}$. For a meterstick rotating around its end, $I=\frac{1}{3} m \ell^2$ and $mgh/I=(10)(\ell/2)/ (1/3) \ell^2 = 30/2=15 s^{-2}$. So $T_{ideal}=2\pi/\sqrt{15}=1.622$.

In [0]:
def fpend(y,t):
    yprime = np.zeros_like(y)
    yprime[0] = y[1]
    yprime[1] = -15 * np.sin(y[0])
    return yprime
y0 = [math.pi/4.0, 0]

t = np.linspace(0,3)
ypend = vsolve_rk4(fpend, t, y0)
plt.plot(t, ypend[:,0], "-og", ms = 3)
plt.plot([2*math.pi/math.sqrt(15)]*10, np.linspace(-.8,.8,10), "r")
plt.annotate("$T_{ideal} = 2\pi/\sqrt{15}$", (1.622,.8), (0.5,0.8), arrowprops = {"arrowstyle":'->'})

###Physical pendulum vs the "ideal" $\sin\theta \approx \theta$ approximation

In the physical pendulum, the torque is proportional to $\sin\theta$ and acts as a restoring force. The difference in torque between the physical and ideal pendula is then proportional to:

$$ \sin\theta - \theta = -\frac{\theta^3}{3!} + \frac{\theta^5}{5!} + \mathcal{O}(\theta^7) $$

In class, we used $\theta_0=\pi/4$ and released the meterstick from rest, so $\theta_0$ is the maximum amplitude.  Plugging $\theta_0$ into the expansion, shows that $\theta>\sin\theta$ for all  $\theta\leq\theta_0$. Thus, since the restoring force is smaller, we expect the physical pendulum has a longer period, as observed.

Note, this is even a bigger effect if one chooses a larger $\theta_0$. 

###Air drag on a projectile

The drag force on an object like a baseball can be written as:

$$F_D = \frac{1}{2} \rho C_D A v^2$$

Where $C_D\approx \frac{1}{2}$ is the drag coefficient, $\rho$ is the density of air, $A$ is the cross-sectional area of the baseball and $v$ is the velocity. 

Consider throwing the ball vertically. Newton's 2nd law tells us:

$$y'' = -g + \frac{\rho C_D A}{m} \frac{v^3}{|v|}$$

Plugging in numbers I found $\frac{\rho C_D A}{m}=0.01 m^{-1}$. The differential equation can then be solved as before. As usual, I plot the analytical solution to the problem, or a related problem, as a check. In this case, if we just set $C_D=0$ the calculation should agree with the familiar parabolic equation.

In [0]:
def f1d_drag(y,t):
    yprime = np.zeros_like(y)
    yprime[0] = y[1]
    yprime[1] = 0.01 * (-np.sign(y[1])) * y[1]**2 - 10
    return yprime

f1d_nodrag = lambda x0, v0, a, t: x0 + v0*t + 0.5*a*t**2

y0 = [0,40]
t = np.linspace(0,8)
plt.plot(t, f1d_nodrag(0, 40, -10, t))
ydrag = vsolve_rk4(f1d_drag, t, y0)
plt.plot(t, ydrag[:,0], "ro", ms = 3)

##Using `scipy.integrate.odeint`

In [0]:
import scipy.integrate as ig
yodeint = ig.odeint(f1d_drag, y0, t)
plt.plot(t, f1d_nodrag(0, 40, -10, t))
plt.plot(t, ydrag[:,0], '-y', ms = 3)
plt.plot(t, yodeint[:,0], 'ro', ms = 3)

In [0]:
plt.plot(t, ydrag[:,0] - yodeint[:,0])